<a href="https://colab.research.google.com/github/chefPony/nn_zero_to_hero/blob/master/gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Transformer from scratch

## Load Data

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-07 09:09:11--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-09-07 09:09:11 (25.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [69]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for c, i in stoi.items()}

encode = lambda x: [stoi[c] for c in x]
decode = lambda x: [itos[i] for i in x]
print(vocab_size)
print("".join(chars))
print(itos)
print(stoi)

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b

In [70]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

encoded_data = torch.tensor([stoi[c] for c in text])
print(encoded_data[:1000])

train_data = encoded_data[:int(len(text) * 0.9)]
val_data = encoded_data[int(len(text) * 0.9):]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [71]:
torch.manual_seed(42)

def get_batch(data: torch.Tensor, batch_size: int, block_size: int):
  ix = torch.randint(low=0, high=data.shape[0] - block_size, size=(batch_size, 1))
  xb = torch.stack([data[i : i+block_size] for i in ix], dim=0)
  yb = torch.stack([data[i+1 : i+block_size+1] for i in ix], dim=0)
  return xb, yb

xb, yb = get_batch(encoded_data, 4, 8)
print(xb.shape)
print(yb.shape)
print("batch")
print(xb)
print("target")
print(yb)

torch.Size([4, 8])
torch.Size([4, 8])
batch
tensor([[42,  1, 58, 46, 59, 57,  1, 21],
        [54, 56, 47, 43, 57, 58, 11,  0],
        [49, 47, 52, 45, 12,  1, 58, 46],
        [58, 46, 53, 59, 58,  1, 56, 43]])
target
tensor([[ 1, 58, 46, 59, 57,  1, 21,  1],
        [56, 47, 43, 57, 58, 11,  0, 37],
        [47, 52, 45, 12,  1, 58, 46, 53],
        [46, 53, 59, 58,  1, 56, 43, 42]])


## Baseline: Bigram Model

In [75]:
import torch.nn as nn
from torch.nn import functional as F

class BigramModel(nn.Module):

  def __init__(self, vocab_size: int):
    super().__init__()
    self.logits = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)


  def forward(self, x, targets=None):
    logits = self.logits(x) # (B, T, C)
    if targets is None:
      loss = None
    else:
       B, T, C = logits.shape
       loss = F.cross_entropy(logits.view(B * T, C), targets.view(B * T))
    return logits, loss

  @torch.no_grad
  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)  #(B, n_vocab)
      probs = F.softmax(logits, dim=-1)[:, -1, :]
      next_idx = torch.multinomial(probs, 1)
      idx = torch.cat([idx, next_idx], dim=1)
    return idx


xb, yb = get_batch(train_data, batch_size=8, block_size=2)
bigram = BigramModel(vocab_size)
print(bigram(xb, yb))
print("".join(decode(bigram.generate(torch.tensor([[0 , 0]]), 100).squeeze().tolist())))

(tensor([[[-0.2904,  0.2590,  0.7079,  ...,  0.7628,  1.8667, -1.1203],
         [-0.4843,  0.9146,  1.1201,  ...,  0.6532, -0.1009, -0.8900]],

        [[ 0.8944, -0.6316, -0.3824,  ..., -0.5481, -0.2681, -0.7464],
         [ 0.0475, -0.4170, -0.5255,  ...,  1.3355,  0.6254, -0.4938]],

        [[-0.8011,  1.0484, -0.2927,  ..., -0.0139,  0.0979, -2.2571],
         [ 0.9068,  2.1892,  1.7476,  ...,  0.3176, -1.0368, -0.7474]],

        ...,

        [[ 1.3910,  0.9389,  0.0155,  ...,  0.0100, -0.5601, -1.3799],
         [ 0.1100,  1.6044,  0.1107,  ..., -1.1784,  0.5426,  0.3707]],

        [[-0.4843,  0.9146,  1.1201,  ...,  0.6532, -0.1009, -0.8900],
         [-1.0480,  0.0925,  0.6995,  ..., -1.3171, -2.0305,  1.4573]],

        [[ 0.6370,  0.4737, -1.3896,  ..., -0.7042, -2.9895, -0.5418],
         [-0.6855,  2.2579,  0.7349,  ..., -1.6794,  0.0107,  0.0091]]],
       grad_fn=<EmbeddingBackward0>), tensor(4.4246, grad_fn=<NllLossBackward0>))


&xG.gbXlFFCzWpUVAXDhgL
R PvR3GzcB$sUz

In [78]:
lr = 1.
block_size = 2
batch_size = 16
n_iter = 20000
model = BigramModel(vocab_size)

@torch.no_grad
def evaluate_model(model, batch_size, block_size, num_batches=100):
  loss_tr, loss_va = 0, 0
  for _ in range(num_batches):
    xb_tr, yb_tr = get_batch(train_data, batch_size, block_size)
    xb_va, yb_va = get_batch(val_data, batch_size, block_size)
    _, lossb_tr = model(xb_tr, yb_tr)
    _, lossb_va = model(xb_va, yb_va)
    loss_tr += lossb_tr
    loss_va += lossb_va
  return loss_tr/num_batches, loss_va/num_batches

for i in range(n_iter):
  xb, yb = get_batch(train_data, batch_size, 2)
  _, loss = model(xb, yb)

  for p in model.parameters():
    p.grad = None

  loss.backward()

  for p in model.parameters():
    p.data += -lr * p.grad

  if i % 1000 == 0:
    loss_tr, loss_va = evaluate_model(model, batch_size, block_size)
    print(f"Train: {loss_tr:.4f} Validation: {loss_va:.4f}")

Train: 4.6459 Validation: 4.6332
Train: 2.9655 Validation: 2.9123
Train: 2.7739 Validation: 2.7765
Train: 2.6468 Validation: 2.6758
Train: 2.6503 Validation: 2.6450
Train: 2.6039 Validation: 2.6303
Train: 2.5625 Validation: 2.6065
Train: 2.5952 Validation: 2.6100
Train: 2.5765 Validation: 2.6054
Train: 2.5661 Validation: 2.5468
Train: 2.5249 Validation: 2.5553
Train: 2.5466 Validation: 2.5220
Train: 2.5221 Validation: 2.5160
Train: 2.5155 Validation: 2.5485
Train: 2.5137 Validation: 2.5543
Train: 2.4912 Validation: 2.5097
Train: 2.5491 Validation: 2.5359
Train: 2.5521 Validation: 2.5370
Train: 2.4920 Validation: 2.5409
Train: 2.4892 Validation: 2.5389


In [80]:
print("".join(decode(model.generate(torch.tensor([[0 , 0]]), 100).squeeze().tolist())))




MI y I t ndier:
B3t te iste ire?
HLAn ayor fe, y ir towean Whee withmirswan, engrird wiowiconthance
